In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df

caso = 'julieta'
df = df_caso(caso)

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

34 rows found with non string elements for column comment (0.67%)
Deleting 255 columns for which max target value is over 7 (5.00%)
4806 available rows after processing


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.2, stratify=df['sel'], random_state=0)

In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

In [7]:
%%time
# Perform topic modeling using HDP
hdp_model = gensim.models.HdpModel(doc_term_matrix, id2word=dictionary)

CPU times: user 9.02 s, sys: 2.08 s, total: 11.1 s
Wall time: 7.32 s


In [8]:
# Print the generated topics
topics = hdp_model.print_topics()
for topic in topics:
    print(topic)

(0, '0.020*si + 0.015*información + 0.010*beca + 0.010*Julieta + 0.009*usar + 0.009*debería + 0.007*situación + 0.007*copiar + 0.007*puede + 0.006*caso')
(1, '0.014*si + 0.010*información + 0.009*beca + 0.009*Julieta + 0.006*situación + 0.006*debería + 0.006*copiar + 0.005*usar + 0.005*puede + 0.005*caso')
(2, '0.011*si + 0.009*beca + 0.008*Julieta + 0.008*información + 0.007*debería + 0.005*copiar + 0.005*puede + 0.004*ser + 0.003*control + 0.003*(')
(3, '0.009*si + 0.009*información + 0.007*beca + 0.007*Julieta + 0.006*debería + 0.005*usar + 0.005*copiar + 0.005*podría + 0.004*puede + 0.003*respuesta')
(4, "0.009*debería + 0.008*podría + 0.008*Si + 0.007*perder + 0.007*' + 0.007*wsp + 0.007*beca + 0.005*ser + 0.005*caso + 0.004*futuro")
(5, '0.008*si + 0.006*Julieta + 0.006*beca + 0.006*información + 0.005*debería + 0.004*puede + 0.004*usar + 0.003*copiar + 0.003*control + 0.003*respuesta')
(6, '0.007*si + 0.006*beca + 0.005*Julieta + 0.005*información + 0.004*debería + 0.003*copiar 

In [9]:
print(' '.join(tokenized_test[0]))

Si hace piola cambia desarrollo ira sumario asi puede hacer asi puede tener buena nota despues algun tiempo libre estudia bien


In [21]:
# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = hdp_model[doc_term_matrix]
for tup in topic_probs:
    print(tup)

[(0, 0.9591445143380576)]


## Visualización

In [23]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [26]:
vis_data = gensimvis.prepare(hdp_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)